## This Script shows how to use the conversion scripts to convert AMASS npz data into a Unity-Friendly JSON file.

Note that these scripts were written in python 3.8, and therefore probably require at least python 3.6 to run. 
You'll need an up-to-date version of scipy package since older versions do not include the scipy.spatial.transform module. 

Requirements:
* scipy
* numpy
* json

Run This cell to make sure you have dependencies installed.


In [1]:
# Check dependencies
import h5py
import version_check_util as vers
import scipy
vers.check_version("scipy", scipy.__version__, "1.4.1")
import numpy
vers.check_version("numpy", numpy.__version__, "1.18.1")
import json
vers.check_version("json", json.__version__, "2.0.9")

# Internal Imports
from AmassData import AMASSDataConverter


scipy version 1.4.1, good
numpy version 1.18.2, good
json version 2.0.9, good


# Convert SMPL-H AMASS animation .npz data files to Unity:


Point to smpl-h npz animation file


In [2]:
npz_data_filename = 'AMASS_Data/punching_poses.npz'


Create converter


In [3]:
converter = AMASSDataConverter(npz_data_filename)


gender: female
betas: (16,)
poses: (234, 156)
dmpls: (234, 8)
trans: (234, 3)
frames detected: 234
fps: 60


Write to output json (full path can be given. this example puts it in the current folder)


In [4]:
h5_name = 'test_data.h5'
converter.write_to_h5(h5_name)


poses: (234, 156)
gender: ()
mocap_framerate: ()
betas: (16,)
marker_data: (234, 86, 3)
dmpls: (234, 8)
marker_labels: (86,)
trans: (234, 3)

	*** DONE CONVERSION ***



In [23]:
from scipy.spatial.transform import Rotation
with h5py.File(h5_name, 'r') as hf:
    print(hf.keys())
    J = hf['poses']
    J = numpy.reshape(J, [J.shape[0], 52, 3])
    print(J.shape)
    print(J.dtype)
    #print(J[2])
    #print(J[200])
    print("**")
    test = J[118][39]
    print(f"test: { test }")
    #for i in range(0, len(J)):
        #print(f"frame {i}: {J[i]}")
    rotvec = Rotation.from_rotvec(test)
    print(Rotation.as_quat(rotvec))


<KeysViewHDF5 ['betas', 'dmpls', 'gender', 'marker_data', 'marker_labels', 'mocap_framerate', 'poses', 'trans']>
(234, 52, 3)
float64
**
test: [-0.09639297  0.09091566  0.18845929]
[-0.04808997  0.04535737  0.0940214   0.99337306]


In [26]:
with h5py.File(h5_name, 'r') as hf:
    print(hf.keys())
    for key in hf.keys():
        print(f"data: {key}: shape: {hf[key].shape}")

<KeysViewHDF5 ['betas', 'dmpls', 'gender', 'marker_data', 'marker_labels', 'mocap_framerate', 'poses', 'trans']>
data: betas: shape: (16,)
data: dmpls: shape: (234, 8)
data: gender: shape: ()
data: marker_data: shape: (234, 86, 3)
data: marker_labels: shape: (86,)
data: mocap_framerate: shape: ()
data: poses: shape: (234, 156)
data: trans: shape: (234, 3)


Second data conversion example


In [5]:
sample_data = "AMASS_Data/amass_sample.npz"
converter = AMASSDataConverter(sample_data)
converter.write_to_json('amass_sample.json')


120.0
gender: male
betas: (16,)
poses: (601, 156)
dmpls: (601, 8)
trans: (601, 3)
frames detected: 601
fps: 120


For batch runs, you can suppress most messages by adding show_messages=False when creating the converter.

In [ ]:
converter = AMASSDataConverter(npz_data_filename, show_messages=False)

You can also call these scripts from the command line using (assuming you're in the right folder)

`python AmassData.py 'Path/To/npzFile.npz' 'Path/to/destination/jsonFile.json'`